In [7]:
import graphs.py
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import seaborn as sns
import matplotlib.pyplot as plt


SyntaxError: invalid syntax (graphs.py, line 1)

In [4]:
def merge_and_load_datasets(name_1, name_2, name_3):
    """
    Объединяет датасеты и загружает данные из других баз.

    Args:
        name_1, name_2, name_3 (str): Имена CSV-файлов (датасетов).

    Returns:
        pandas.DataFrame: Объединенный датасет.
    """
    data = pd.read_csv(name_1)
    bacterial_descriptors = pd.read_csv(name_2)
    drug_descriptors = pd.read_csv(name_3)

    merged_data = pd.merge(data, bacterial_descriptors, on='Bacteria', how='left')
    merged_data = pd.merge(merged_data, drug_descriptors, on='drug', how='left')

    return merged_data


def calculate_rdkit_descriptors(s, mol_objects):
    """
    Вычисляет дескрипторы RDKit для заданных молекул.

    Args:
        s (str): Имя дескриптора для вычисления.
        mol_objects (list): Список объектов молекул.

    Returns:
        list: Список значений дескрипторов.
    """
    descriptors = []
    for mol in mol_objects:
        if mol is not None:
            # Вычисление указанного дескриптора для молекулы
            descriptors += [eval(s)(mol)]
        else:
            descriptors += [np.NaN]
    return descriptors


def process_data(data):
    """
    Обрабатывает и очищает базу данных.

    Args:
        data (pandas.DataFrame): Объединенный датасет.

    Returns:
        pandas.DataFrame: Обработанный датасет.
    """
    # Преобразование значений в столбце 'gram' на числовые (0 и 1)
    data['gram'] = data['gram'].replace({'n': 0, 'p': 1})

    ZOI = ['ZOI_drug', 'ZOI_NP', 'ZOI_drug_NP']
    for zoi in ZOI:
        # Удаление символа "+" и всего, что следует за ним, в столбцах 'ZOI_drug', 'ZOI_NP' и 'ZOI_drug_NP'
        data[zoi] = data[zoi].str.replace(r'\+.*', '', regex=True)

    # Исключение строк, где значение в столбце 'kingdom' равно 'Fungi'
    data = data[data['kingdom'] != 'Fungi']

    # Исключение строк, где значение в столбце 'NP_concentration' содержит символ '/'
    data = data[~data['NP_concentration'].str.contains('/', na=False)]

    columns = ['Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y',
               'NP size_min', 'NP size_max',
               'min_Incub_period, h', 'avg_Incub_period, h', 'max_Incub_period, h',
               'growth_temp, C', 'prefered_name', 'kingdom',
               'subkingdom', 'clade', 'phylum', 'class',
               'order', 'family', 'genus', 'species']

    # Удаление указанных столбцов
    data = data.drop(columns, axis=1)

    return data


def analyze_data(data):
    """
    Анализирует данные и строит графики.

    Args:
        data (pandas.DataFrame): Обработанный датасет.
    """
    # Выбор числовых столбцов
    numeric_data = data.select_dtypes(include='number')

    # Построение тепловой карты корреляции
    graphs.matrix_correlation(numeric_data)
    graphs.histogram(data, 'NP size_avg')


def save_data(data, name = 'data_new.csv'):
    """
    Сохраняет обработанные данные в CSV-файл и выводит информацию о датасете.

    Args:
        data (pandas.DataFrame): Обработанный датасет.
        name (str): Имя CSV-файла для сохранения.
    """
    # Сохранение обработанных данных в CSV-файле
    data.to_csv(name, index=False)

    # Вывод информации о датасете
    print(data.info())

    # Вывод процента пропущенных значений в датасете
    print(data.isnull().mean() * 100)


def main():
    # Объединение датасетов и загрузка данных
    data = merge_and_load_datasets('data.csv', 'bacterial_descriptors.csv', 'drug_descriptors.csv')

    # Получение SMILES-строк и объектов молекул
    smiles = data['smiles'].tolist()
    mol_objects = [Chem.MolFromSmiles(smi) if isinstance(smi, str) else None for smi in smiles]

    # Вычисление дескрипторов RDKit
    descriptors = ['Descriptors.MolWt', 'Descriptors.MolLogP', 'Descriptors.NumHAcceptors',
                   'Descriptors.NumHDonors', 'Descriptors.TPSA']
    for descriptor in descriptors:
        descriptor_values = calculate_rdkit_descriptors(descriptor, mol_objects)
        data[descriptor.split('.')[-1]] = descriptor_values

    # Обработка и очистка данных
    data = process_data(data)

    # Анализ данных и построение графиков
    analyze_data(data)

    # Сохранение данных и вывод информации о датасете
    save_data(data, 'merged_data.csv')

if __name__ == '__main__':
    main()

NameError: name 'graphs' is not defined

In [100]:

old_names = merged_data.drug.unique()
wanted_names = ['Amoxicillin', 'Penicillin', 'Vancomycin', 'Gentamicin',
       'Ceftazidime', 'Ampicillin', 'Faropenem', 'Ceftriaxone',
       'Rifampicin', 'Azithromycin', 'Chloramphenicol', 'Oxytetracycline',
       'Piperacillin', 'Tetracycline', 'Ciprofloxacin', 'Cefuroxime',
       'Streptomycin', 'Co-trimoxazole', 'Oxacillin', 'Cefotaxime',
       'Fosfomycin', 'Mupirocin', 'Neomycin', 'Doxycycline',
       'Nitrofurantoin', 'Erythromycin', 'Cefazolin', 'Cefixime ',
       'Carbenicillin ', 'Amikacin', 'Trimethoprim', 'Kanamycin',
       'Levofloxacin', 'Imipenem', 'Nalidixic acid', 'Fusidic acid ',
       'Ciprofloxacin', 'Polymyxin', 'Cephalexin ', 'Cefepime',
       'Amphotericin B', 'Clindamycin ', 'Colistin ', 'Biapenem',
       'Aztreonam', np.nan]
def rename_drugs(old_names, wanted_names):
    for i in range(len(wanted_names)):
        if wanted_names[i] != old_names[i]:
            np.where(merged_data.drug == old_names[i], wanted_names[i], merged_data.drug)

rename_drugs(old_names, wanted_names)
merged_data.drug.unique()

array(['Amoxicillin', 'Penicillin', 'Vancomycin', 'Gentamicin',
       'Ceftazidime', 'Ampicillin', 'Faropenem', 'Ceftriaxone',
       'Rifampicin', 'Azithromycin', 'Chloramphenicol', 'Oxytetracycline',
       'Piperacillin', 'Tetracycline', 'ciprofloxacin', 'Cefuroxime',
       'Streptomycin', 'Co-trimoxazole', 'Oxacillin', 'Cefotaxime',
       'Fosfomycin', 'mupirocin', 'Neomycin', 'Doxycycline',
       'Nitrofurantoin', 'Erythromycin', 'cefazolin', 'Cefixime ',
       'Carbenicillin ', 'Amikacin', 'Trimethoprim', 'Kanamycin',
       'Levofloxacin', 'Imipenem', 'Nalidixic acid', 'Fusidic acid ',
       'Ciprofloxacin', 'Polymyxin', 'Cephalexin ', 'Cefepime',
       'Amphotericin B', 'Clindamycin ', 'colistin ', 'Biapenem',
       'Aztreonam', nan], dtype=object)

In [73]:
all_drugs = pd.read_csv('drugbank.csv')


In [88]:
drugsfilt = all_drugs.name.isin(merged_data.drug.unique())
drugs = all_drugs[drugsfilt]
drugs

,Unnamed: 0,type,drugbank_id,name,state,cal_logp,cal_logs,cal_water_solubility,cal_molecular_weight,cal_polar_surface_area,...,cal_pka_acidic,cal_pka_basic,cal_physiological_charge,cal_number_of_rings,cal_bioavailability,cal_rule_of_five,exp_molecular_weight,exp_logp,exp_pka,half_life
0,0,biotech,DB00001,Lepirudin,liquid,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6963.425,NaN,NaN,Approximately 1.3 hours
1,1,biotech,DB00002,Cetuximab,liquid,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,145781.600,NaN,NaN,After administration of a 400 mg/m<sup>2</sup>...
2,2,biotech,DB00003,Dornase alfa,liquid,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,29253.900,NaN,NaN,NaN
3,3,biotech,DB00004,Denileukin diftitox,liquid,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,57647.300,NaN,NaN,70-80 min
4,4,biotech,DB00005,Etanercept,liquid,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,51234.900,NaN,NaN,Etanercept has a mean half-life of elimination...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14589,14589,small molecule,DB16742,RP-67580,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14590,14590,small molecule,DB16743,Nolpitantium chloride,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14591,14591,small molecule,DB16744,CP-96345,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14592,14592,small molecule,DB16745,PXT 3003,NaN,4.74,-4.5,1.15e-02 g/l,353.466,63.25,...,18.99,4.77,0.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN
